In [101]:
import numpy as np
import dolfinx
from mpi4py import MPI
import ufl
import sympy 

In [102]:
domain = dolfinx.mesh.create_rectangle(MPI.COMM_WORLD,[[-1,-1],[1,1]],n=[10,10])

In [3]:
N = 2
M = 4
V = dolfinx.fem.TensorFunctionSpace(domain,('CG',1),shape=(N,M))
V_interp= dolfinx.fem.FunctionSpace(domain,('CG',1))

In [4]:
F = dolfinx.fem.Function(V_interp)
F.function_space

FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), FiniteElement('Lagrange', triangle, 1))

In [5]:
def l_f(x):
    #print(x)
    #print(x.shape)
    return x[0]
F.interpolate(l_f)
F


Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), FiniteElement('Lagrange', triangle, 1)), 0)

In [6]:
8*600

4800

In [7]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

X = ufl.SpatialCoordinate(V)

In [8]:
from ufl import Dx, dX


vars = {'m':None,'g':None}
ufl_constants = {k:dolfinx.fem.Constant(V,c=np.complex128(1) if v is None else v) for k,v in vars.items()}


In [9]:
dolfinx.fem.Constant(V,np.complex128(1))

Constant(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), (), 2)

In [10]:

i,j,k = ufl.indices(3)
H = -0.5/ufl_constants['m']*ufl.grad(u)[i,j,k]*ufl.grad(ufl.conj(v))[i,j,k]*ufl.dx+ 0.5*ufl_constants['g']*ufl.conj(v)[i,j]*u[i,j]*F*ufl.dx

In [11]:
ufl_constants 
H

Form([Integral(IndexSum(IndexSum(IndexSum(Product(Indexed(Grad(Conj(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 0, None))), MultiIndex((Index(8), Index(9), Index(10)))), Product(Indexed(Grad(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 1, None)), MultiIndex((Index(8), Index(9), Index(10)))), Division(FloatValue(-0.5), Constant(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), (), 0)))), MultiIndex((Index(8),))), MultiIndex((Index(9),))), MultiIndex((Index(10),))), 'cell', Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), 'everywhere', {}, None), Integral(Product(Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), FiniteElement('Lagrange', t

In [12]:
L = dolfinx.fem.form(H)

In [13]:
L

In [14]:
A=dolfinx.fem.petsc.assemble_matrix(L)

In [15]:
A.assemble()
import scipy
from scipy.sparse import csr_matrix
a = csr_matrix(A.getValuesCSR()[::-1])

In [16]:
a.todense()

matrix([[-0.99586667+0.j,  0.        +0.j,  0.        +0.j, ...,
          0.        +0.j,  0.        +0.j,  0.        +0.j],
        [ 0.        +0.j, -0.99586667+0.j,  0.        +0.j, ...,
          0.        +0.j,  0.        +0.j,  0.        +0.j],
        [ 0.        +0.j,  0.        +0.j, -0.99586667+0.j, ...,
          0.        +0.j,  0.        +0.j,  0.        +0.j],
        ...,
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j, ...,
         -0.5016    +0.j,  0.        +0.j,  0.        +0.j],
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j, ...,
          0.        +0.j, -0.5016    +0.j,  0.        +0.j],
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j, ...,
          0.        +0.j,  0.        +0.j, -0.5016    +0.j]])

In [17]:
M=ufl_constants['m']

In [18]:
import sympy
g = sympy.symbols('g')
c = np.complex128

In [19]:
n = sympy.symbols('n')
syms= {n:1}

In [20]:
(1,2)+(2,3)

(1, 2, 2, 3)

In [21]:

A = [[c(1),c(2),c(3)],[0,1,2],[0,3,3.0*g]]
A=sympy.Array(A)


In [22]:
def func1(x):
    print(1,x)
    return(1-x)

sympy.lambdify(g,A.tolist())(func1(1))

1 1


[[1.0, 2.0, 3.0], [0, 1, 2], [0, 3, 0.0]]

In [23]:
ufl.grad(u),

(Grad(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 1, None)),)

In [24]:
F=v*u
type(F)

ufl.tensors.ComponentTensor

In [25]:
print(isinstance(F,ufl.algebra.Product))
F.ufl_operands


False


(IndexSum(Product(Indexed(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 0, None), MultiIndex((Index(11), Index(13)))), Indexed(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 1, None), MultiIndex((Index(13), Index(12))))), MultiIndex((Index(13),))),
 MultiIndex((Index(11), Index(12))))

In [26]:
type(ufl.grad(u))

ufl.differentiation.Grad

In [27]:
def k_func(uv:ufl.algebra.Product,i):
    #argument is either u*ufl.conj(v) or terms like this with derivatives
    u_func,v_func = uv.ufl_operands
    if not isinstance(u_func,ufl.differentiation.Grad):
        return -1j*ufl.grad(u_func)*v
    else:
        return u_func* 1j*ufl.grad(v)
    

In [28]:
kx,ky,kz = sympy.symbols('k_x,k_y,k_z')

m,g = sympy.symbols('m,g')

F = sympy.Function('f')
A= sympy.Array([[1*kx*kx+ky*kz+ky*m,0,0],[0,2*ky*g,0],[1,0,3*kz]])
ten_shape = A.shape
spatial_dim = 3
A

[[k_x**2 + k_y*k_z + k_y*m, 0, 0], [0, 2*g*k_y, 0], [1, 0, 3*k_z]]

In [29]:
disassemble_dict = {}
from sympy import degree_list,degree
import numpy as np
for i,val in enumerate(np.array(A).ravel()):
    # split up value into terms 
    loop_terms = (val,) if not isinstance(val,sympy.core.add.Add) else val.args
    for term in loop_terms:
        try:
            orders = degree_list(term,(kx,ky,kz))
        except TypeError as e:
            continue 
        print(orders)
        arr = disassemble_dict.get(sum(orders),np.zeros(ten_shape+(3,)*(sum(orders)),'O'))
        # k-s commute, and by convention we will always order them as k_x k_x ... k_x k_y k_y ... k_y k_z k_z ... k_z
        arr_i = np.unravel_index(i,ten_shape)+(0,)*orders[0]+(1,)*orders[1]+(2,)*orders[2]
        arr[arr_i] += sympy.lambdify((kx,ky,kz),term)(1,1,1)
        disassemble_dict[sum(orders)] = arr

(2, 0, 0)
(0, 1, 1)
(0, 1, 0)
(0, 1, 0)
(0, 0, 0)
(0, 0, 1)


In [30]:
disassemble_dict[2][:,:,2,2]

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=object)

In [31]:
from numpy import isin


val = A[0,0]
type(val)
kx =sympy.symbols(r'k_x')

In [32]:
type(degree(A.tolist()[0][1],kx))

sympy.core.numbers.NegativeInfinity

In [33]:
sympy.symbols('n_{z-modes}')

n_{z-modes}

In [34]:
A/

SyntaxError: invalid syntax (847341840.py, line 1)

In [ ]:
B = sympy.Array([[1*kx*kx+ky*kz+ky,0,0],[0,2*ky,0],[1,0,3*kz]])

In [ ]:
A == B


False

In [ ]:
D = {1:1,2:2,3:3}
DD = {3:3,2:2,1:1}

In [ ]:
D == DD

True

In [ ]:
ufl_constants

{'m': Constant(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), (), 0),
 'g': Constant(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), (), 1)}

In [ ]:
A

[[k_x**2 + k_y*k_z + k_y*m, 0, 0], [0, 2*g*k_y, 0], [1, 0, 3*k_z]]

In [ ]:
#A.subs({s: ufl_constants[s.name] for s in (m,g)})

In [ ]:
R=sympy.lambdify((m,g,kx,ky,kz),A)(ufl_constants['m'],ufl_constants['g'],1,1,1)

In [ ]:
ufl_constants['m']

Constant(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), (), 0)

In [ ]:
R[0,0] 


A = np.arange(16).reshape(4,4).astype(np.complex128)

tA = ufl.as_tensor(A*x[0]*x[1]**2)
print(tA)

[
  [0, x[0] * x[1] ** 2, 2.0 * x[0] * x[1] ** 2, 3.0 * x[0] * x[1] ** 2],
  [4.0 * x[0] * x[1] ** 2, 5.0 * x[0] * x[1] ** 2, 6.0 * x[0] * x[1] ** 2, 7.0 * x[0] * x[1] ** 2],
  [8.0 * x[0] * x[1] ** 2, 9.0 * x[0] * x[1] ** 2, 10.0 * x[0] * x[1] ** 2, 11.0 * x[0] * x[1] ** 2],
  [12.0 * x[0] * x[1] ** 2, 13.0 * x[0] * x[1] ** 2, 14.0 * x[0] * x[1] ** 2, 15.0 * x[0] * x[1] ** 2]
]


In [ ]:
np.array([a*x[0] for a in A.ravel()]).reshape(A.shape)

array([[Zero((), (), ()),
        Indexed(SpatialCoordinate(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0)), MultiIndex((FixedIndex(0),))),
        Product(FloatValue(2.0), Indexed(SpatialCoordinate(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0)), MultiIndex((FixedIndex(0),)))),
        Product(FloatValue(3.0), Indexed(SpatialCoordinate(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0)), MultiIndex((FixedIndex(0),))))],
       [Product(FloatValue(4.0), Indexed(SpatialCoordinate(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0)), MultiIndex((FixedIndex(0),)))),
        Product(FloatValue(5.0), Indexed(SpatialCoordinate(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0)), MultiIndex((FixedIndex(0),)))),
        Product(FloatValue(6.0), Indexed(SpatialCoordinate(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0)), MultiIndex((FixedIndex(0),)))),
        Product(Float

In [ ]:
x = ufl.SpatialCoordinate(V)
print(tA*x[0])

{ A | A_{i_{52}, i_{53}} = x[0] * ([
  [0, 1.0, 2.0, 3.0],
  [4.0, 5.0, 6.0, 7.0],
  [8.0, 9.0, 10.0, 11.0],
  [12.0, 13.0, 14.0, 15.0]
])[i_{52}, i_{53}] }


In [ ]:

u= ufl.TrialFunction(V)
v = ufl.TestFunction(V)
I = ufl.indices(2)
u[I].dx(1)*ufl.conj(v)[I]*ufl.dx

Form([Integral(IndexSum(IndexSum(Product(Indexed(Grad(Indexed(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 1, None), MultiIndex((Index(44), Index(45))))), MultiIndex((FixedIndex(1),))), Indexed(Conj(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 0, None)), MultiIndex((Index(44), Index(45))))), MultiIndex((Index(44),))), MultiIndex((Index(45),))), 'cell', Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), 'everywhere', {}, None)])

In [ ]:
u[0,0]

Indexed(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 1, None), MultiIndex((FixedIndex(0), FixedIndex(0))))

In [ ]:
V

FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={}))

In [ ]:
u.dx(1)*ufl.conj(v)

ComponentTensor(IndexSum(Product(Indexed(ComponentTensor(Indexed(Grad(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 1, None)), MultiIndex((Index(21), Index(22), FixedIndex(1)))), MultiIndex((Index(21), Index(22)))), MultiIndex((Index(23), Index(25)))), Indexed(Conj(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 0, None)), MultiIndex((Index(25), Index(24))))), MultiIndex((Index(25),))), MultiIndex((Index(23), Index(24))))

In [ ]:
def F(x):
    x = np.asarray(x)
    print(x.shape)
    if len(x.shape) == 1:
        x = x[np.newaxis,:]
        print(x.shape)
    x_cords = np.linspace(-10,10,100).astype(np.complex128)
    y_vals = x_cords*10
    return_arr = []
    for xx in x:
        return_arr.append(y_vals[np.argmin(np.abs(x_cords-xx[0]))]) # find closest point to interpolate
    return np.stack(return_arr)

In [ ]:
F([1,0,0])

(3,)
(1, 3)


array([9.09090909+0.j])

In [ ]:
X

SpatialCoordinate(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 3))

In [ ]:
fem_F = dolfinx.fem.Function(V)

In [ ]:
simple_F = lambda x:np.arange(6).reshape(2,3)

In [ ]:
fem_F

Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 3), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(3, 2), symmetry={})), 2)

In [37]:
from mpi4py import MPI
domain = dolfinx.mesh.create_rectangle(MPI.COMM_WORLD,[[-1,-1],[1,1]],n=[25,25])

N = 2
M = 4
V = dolfinx.fem.TensorFunctionSpace(domain,('CG',1),shape=(2,2))
V_interp= dolfinx.fem.FunctionSpace(domain,('CG',1))
boundary_value= 1234e11
    #boundary_value = 0
V.mesh.topology.create_connectivity(V.mesh.topology.dim - 1, V.mesh.topology.dim)
boundary_facets = dolfinx.mesh.exterior_facet_indices(V.mesh.topology)
boundary_dofs = dolfinx.fem.locate_dofs_topological(V, V.mesh.topology.dim - 1, boundary_facets)
boundary_state =np.full((2,2),boundary_value, dtype='complex')
infinity_boundary = dolfinx.fem.dirichletbc(boundary_state, boundary_dofs, V)
u_boundary = dolfinx.fem.Function(V)
dolfinx.fem.petsc.set_bc(u_boundary.vector, [infinity_boundary])
u_boundary.x.scatter_forward()
from scipy.sparse import diags
INF= diags(u_boundary.vector.getArray(),offsets=0,format='csr')




In [38]:
import ufl
import dolfinx
import scipy.sparse as spa

In [39]:
Sz = np.array([[1,0],[0,-1]]).astype('complex')
Sx =  np.array([[0,1],[1,0]]).astype('complex')
Sy =   np.array([[0,-1j],[1j,0]]).astype('complex')
uSz = ufl.as_tensor(Sz)
uSx = ufl.as_tensor(Sx)
uSy = ufl.as_tensor(Sy)

SS = ufl.as_tensor(np.stack([Sx,Sy,Sz]))

In [40]:
u,v = ufl.TrialFunction(V),ufl.TestFunction(V)

In [41]:
H = ufl.inner(ufl.grad(u),ufl.grad(v))*ufl.dx
#magnetic z-direction term
i,_,j = ufl.indices(3)
H += np.pi**2*u[i,_]*uSz[i,j]*ufl.conj(v[j,_])*ufl.dx

In [42]:
L=dolfinx.fem.form(H)

In [43]:
l=dolfinx.fem.petsc.assemble_matrix(L)
l.assemble()


SH = ufl.inner(u,v)*ufl.dx
SL = dolfinx.fem.petsc.assemble_matrix(dolfinx.fem.form(SH))
SL.assemble()
S =(spa.csr_matrix(SL.getValuesCSR()[::-1]))


In [50]:
A=(spa.csr_matrix(l.getValuesCSR()[::-1])+INF)


In [51]:
from scipy.sparse.linalg import eigsh
evals,evecs = eigsh(A,M=S,k=10,which='LM',sigma=0)
ev = np.array(evecs[:,0])

In [53]:
evals

array([ 2.55117271,  2.59836618,  2.55117271,  2.59836618, -4.91530664,
       -4.91530664, 10.18010887, 10.18010887, 14.82390216, 14.82390216])

In [63]:
a = evecs[:,0].reshape(int(evecs.shape[0]/2),2)
b = evecs[:,2].reshape(int(evecs.shape[0]/2),2)


fA = dolfinx.fem.Function(V)
fB = dolfinx.fem.Function(V)
fA.x.array[:] = a.flatten()
fB.x.array[:] = b.flatten()

In [64]:
np.einsum('xi,ij,xj',b.conj(),Sz,b)/np.linalg.norm(b)**2

(0.8006199103444824+0j)

In [59]:
np.linalg.norm(a)

12.570411081691798

In [48]:
i,_,j = ufl.indices(3)

E=dolfinx.fem.form(fA[i,_]*uSz[i,j]*ufl.conj(fA[j,_])*ufl.dx)
dolfinx.fem.assemble_scalar(E)




(-0.996445399046172+0j)

In [49]:
h = SS[0,:,i]*fA[i,0]
h2 = SS[0,:,i]*fA[i,1]

In [ ]:
EE = dolfinx.fem.Expression(h,V.element.interpolation_points())
EE2 = dolfinx.fem.Expression(ufl.as_vector((h,h2),V.element.interpolation_points())

INFO:root:running build_ext
INFO:root:building 'libffcx_expressions_e31e8577c4d6593462b06c97d3c87fcedb41d946' extension
INFO:root:/home/oliver/miniconda3/envs/Germanium_Spins_FE/bin/x86_64-conda-linux-gnu-cc -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /home/oliver/miniconda3/envs/Germanium_Spins_FE/include -fPIC -O2 -isystem /home/oliver/miniconda3/envs/Germanium_Spins_FE/include -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/oliver/miniconda3/envs/Germanium_Spins_FE/include -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/oliver/miniconda3/envs/Germanium_Spins_FE/include -fPIC -I/home/oliver/miniconda3/envs/Germanium_Spins_FE/lib/python3.10/site-packages/ffcx/codegeneration -I/home/oliver/miniconda3/envs/Germanium_Spins_FE/include/python3.10 -c libffcx_expressions_e31e8577c4d6593462b06c97d3c87fcedb41d946.c -o ./libffcx_expressions_e31e8577c4d6593462b06c97d3c87fcedb41d

In [ ]:
FEE = dolfinx.fem.Function(V)

FEE.interpolate(EE2)

In [ ]:
_l =np.array(fA.x.array)
_ll =np.array(FEE.x.array)
_l @ _ll

(-64.9431783844471+0j)

In [ ]:
EE.Array

AttributeError: 'Expression' object has no attribute 'Array'

In [ ]:
np.diff(evals)

array([0.04777997, 2.38785828, 0.01518236, ..., 0.03125   , 0.03125   ,
       0.015625  ])

In [ ]:
evals

array([1.57757699e+00, 1.62535696e+00, 4.01321523e+00, ...,
       1.23400000e+14, 1.23400000e+14, 1.23400000e+14])

In [1]:
import os,sys
src_path = '/mnt/c/Users/olisc/NQCP/NQCP_FEM/'
sys.path.append(src_path)

import nqcpfem
from nqcpfem import band_model,fenics,envelope_function

In [2]:
domain = envelope_function.RectangleDomain(2,2,2)
domain.resolution = [35,35]
bm = band_model.BandModel(sympy.Array(np.arange(2*2*1*1).reshape(2,2,1,1)),spatial_dim=2)
fem = fenics.FEniCsModel(bm,domain,0,('CG',1))

NameError: name 'sympy' is not defined

In [ ]:
fem.mesh()
bm.tensor_shape[::2]

sympy.core.containers.Tuple

In [ ]:
tuple(int(f) for f in bm.tensor_shape[::2])

(2, 1)

# TESTING
Make Fencis Observable and check that it works for spinor like the use-case done above (making the bandmodel simply be the poisso equation)

In [5]:
# make poisson equatio and
from nqcpfem.symbolic import Kx,Ky
from nqcpfem.band_model import BandModel
from nqcpfem import fenics
from nqcpfem.solvers import PETScSolver
from nqcpfem.envelope_function import RectangleDomain
import sympy
import numpy as np
poisson_spinor = sympy.Array([[Kx**2+Ky**2+np.pi**2,0],[0,Kx**2+Ky**2-np.pi**2]])
bm = BandModel(poisson_spinor,2)

L = 1e-6
box_domain = RectangleDomain(L, L,L)
box_domain.resolution = [75, 75]

# use domain from setup
model = fenics.FEniCsModel(bm,box_domain,0,('CG',1))

# solve the model and obtain eigenstates in the usual way.
solver = PETScSolver(which='SM',sigma=0,k=10)

eigvals,eigvecs = solver.solve(model)

# take lowest eigenstates and compute the MEL of sigma_z operator

sigma_z = sympy.Array([[1,0],[0,-1]])
Oz = model.construct_observable(sigma_z)
Oz_proj = [Oz.mel(ev1,ev2) for ev1 in eigvecs[:2] for ev2 in eigvecs[:2]]


9.86960440108936 + k_{x}**2 + k_{y}**2
<class 'sympy.core.add.Add'>
0
<class 'sympy.core.numbers.Zero'>
0
<class 'sympy.core.numbers.Zero'>
-9.86960440108936 + k_{x}**2 + k_{y}**2
<class 'sympy.core.add.Add'>
1
<class 'sympy.core.numbers.One'>
0
<class 'sympy.core.numbers.Zero'>
0
<class 'sympy.core.numbers.Zero'>
-1
<class 'sympy.core.numbers.NegativeOne'>


In [26]:
Oz_proj
Oz.mel(eigvecs[1])

0.9999998817415366

In [6]:
np.einsum('xi,ij,xj',eigvecs[0].conj().reshape((-1,2)),np.array(sigma_z).astype(complex),eigvecs[0].reshape((-1,2)))/np.linalg.norm(eigvecs[0])**2

(-1.0960600378736768e-09+0j)

In [7]:
import dolfinx
V = model.function_space()
import ufl

fl = dolfinx.fem.Function(V)
fr = dolfinx.fem.Function(V)

In [8]:
def eigentensor_flatten(eigentensor):
    transposition = tuple(range(1,len(eigentensor.shape)))+(0,)
    print(transposition)
    return np.transpose(eigentensor,transposition).flatten()

In [9]:
fl.x.array[:] = eigentensor_flatten(eigvecs[0])
fr.x.array[:] = eigentensor_flatten(eigvecs[0])


(1, 0)
(1, 0)


In [132]:
sigma_z = np.array([[0,-1j],[1j,0]])
ufl_sigma = ufl.as_tensor(np.array(sigma_z).astype(complex))

In [133]:
i,j = ufl.indices(2)
form = fl[i]*ufl_sigma[i,j]*ufl.conj(fr[j])*ufl.dx
A=dolfinx.fem.form(form)

INFO:root:running build_ext
INFO:root:building 'libffcx_forms_dc43bc9fc662d76ff49dfb155bc5c94a9d826db4' extension
INFO:root:/home/oliver/miniconda3/envs/Germanium_Spins_FE/bin/x86_64-conda-linux-gnu-cc -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /home/oliver/miniconda3/envs/Germanium_Spins_FE/include -fPIC -O2 -isystem /home/oliver/miniconda3/envs/Germanium_Spins_FE/include -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/oliver/miniconda3/envs/Germanium_Spins_FE/include -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/oliver/miniconda3/envs/Germanium_Spins_FE/include -fPIC -I/home/oliver/miniconda3/envs/Germanium_Spins_FE/lib/python3.10/site-packages/ffcx/codegeneration -I/home/oliver/miniconda3/envs/Germanium_Spins_FE/include/python3.10 -c libffcx_forms_dc43bc9fc662d76ff49dfb155bc5c94a9d826db4.c -o ./libffcx_forms_dc43bc9fc662d76ff49dfb155bc5c94a9d826db4.o -O2 -g0
INFO

In [134]:
eigvecs[0].shape
model.band_model.tensor_shape[::2]

(2,)

In [136]:
fl.x.array[:] = model.flatten_eigentensors(eigvecs[0])#eigentensor_flatten(eigvecs[0])
fr.x.array[:] = model.flatten_eigentensors(eigvecs[1])#eigentensor_flatten(eigvecs[0])
s=dolfinx.fem.assemble_scalar(A)
s

(0.006936758570747434-0.9999759341233186j)

In [128]:
from nqcpfem import _mu_B
OS = model.construct_observable(sympy.Array([[0,-1j],[1j,0]])*_mu_B)
m_state = 1/np.sqrt(2)*(eigvecs[0]-1j*eigvecs[1])
p_state = 1/np.sqrt(2)*(eigvecs[0]+1j*eigvecs[1])
OS.mel(m_state,p_state)

0
<class 'sympy.core.numbers.Zero'>
-9.27401004952627e-24*I
<class 'sympy.core.mul.Mul'>
9.27401004952627e-24*I
<class 'sympy.core.mul.Mul'>
0
<class 'sympy.core.numbers.Zero'>


2.1448335596071762e-27

In [129]:
Ox = np.array([[0,1j],[1j,0]]).astype(complex)
Ox = ufl.as_tensor(Ox)

i,j = ufl.indices(2)

In [130]:
L= Ox[i,...]*fl[i]

In [100]:
dL=dolfinx.fem.Expression(L,V.element.interpolation_points())

TypeError: float() argument must be a string or a real number, not 'complex'

In [79]:
print(L)

sum_{i_{29}} ({ A | A_{i_{39}} = ({ A | A_{i_{38}} = ([
  [0, 1.0],
  [1.0, 0]
])[i_{29}, i_{38}] })[i_{39}] * f[i_{29}] }) 


In [75]:
res = dolfinx.fem.Function(V)
res.interpolate(dL)
res_a = res.x.array
res_a = res_a/np.linalg.norm(res_a)

In [77]:

flat = model.flatten_eigentensors(eigvecs[1])
flat = flat/np.linalg.norm(flat)
res_a.conj() @ flat


(-0.999975828789954+0.006936704812487477j)

In [ ]:
FOX = model.construct_observable(Ox)


In [17]:
OS.mel(eigvecs[0])

-0.9999998817415308

In [18]:
OS.bilinear_form

Form([Integral(IndexSum(Product(Indexed(Conj(Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2)), 5)), MultiIndex((Index(24),))), IndexSum(Product(Indexed(ListTensor(ListTensor(FloatValue(1.0), Zero((), (), ())), ListTensor(Zero((), (), ()), FloatValue(-1.0))), MultiIndex((Index(23), Index(24)))), Indexed(Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2)), 6), MultiIndex((Index(23),)))), MultiIndex((Index(23),)))), MultiIndex((Index(24),))), 'cell', Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), 'everywhere', {}, None)])

In [19]:
form

Form([Integral(IndexSum(Product(Conj(Indexed(Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2)), 4), MultiIndex((Index(22),)))), IndexSum(Product(Indexed(ListTensor(ListTensor(FloatValue(1.0), Zero((), (), ())), ListTensor(Zero((), (), ()), FloatValue(-1.0))), MultiIndex((Index(21), Index(22)))), Indexed(Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2)), 3), MultiIndex((Index(21),)))), MultiIndex((Index(21),)))), MultiIndex((Index(22),))), 'cell', Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), 'everywhere', {}, None)])

In [20]:
OS.bilinear_form -form

Form([Integral(IndexSum(Product(Indexed(Conj(Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2)), 5)), MultiIndex((Index(24),))), IndexSum(Product(Indexed(ListTensor(ListTensor(FloatValue(1.0), Zero((), (), ())), ListTensor(Zero((), (), ()), FloatValue(-1.0))), MultiIndex((Index(23), Index(24)))), Indexed(Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2)), 6), MultiIndex((Index(23),)))), MultiIndex((Index(23),)))), MultiIndex((Index(24),))), 'cell', Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), 'everywhere', {}, None), Integral(Product(IntValue(-1), IndexSum(Product(Conj(Indexed(Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2)), 4), MultiIndex

In [21]:
print(form)

{ sum_{i_{22}} (conj((f[i_{22}]))) * (sum_{i_{21}} ([
  [1.0, 0],
  [0, -1.0]
])[i_{21}, i_{22}] * f[i_{21}] )  } * dx(<Mesh #0>[everywhere], {})


In [22]:
print(OS.bilinear_form)

{ sum_{i_{24}} (conj((f)))[i_{24}] * (sum_{i_{23}} ([
  [1.0, 0],
  [0, -1.0]
])[i_{23}, i_{24}] * f[i_{23}] )  } * dx(<Mesh #0>[everywhere], {})


In [23]:

Oz_projection

NameError: name 'Oz_projection' is not defined

In [111]:


import dolfinx
from mpi4py import MPI
from petsc4py import PETSc
print(PETSc.ScalarType)

domain = dolfinx.mesh.create_rectangle(MPI.COMM_WORLD,[[-1,-1],[1,1]],n=[10,10])

V = dolfinx.fem.FunctionSpace(domain,('CG',1))

f = dolfinx.fem.Function(V)
o= 1+1j
O = dolfinx.fem.Constant(V,o)
h = O*f

dolfinx.fem.Expression(h,V.element.interpolation_points(),dtype=np.complex128)




INFO:root:running build_ext


<class 'numpy.complex128'>


INFO:root:building 'libffcx_expressions_1893f83fc25ef35f46772ddc459a44d32c1cf974' extension
INFO:root:/home/oliver/miniconda3/envs/Germanium_Spins_FE/bin/x86_64-conda-linux-gnu-cc -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /home/oliver/miniconda3/envs/Germanium_Spins_FE/include -fPIC -O2 -isystem /home/oliver/miniconda3/envs/Germanium_Spins_FE/include -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/oliver/miniconda3/envs/Germanium_Spins_FE/include -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/oliver/miniconda3/envs/Germanium_Spins_FE/include -fPIC -I/home/oliver/miniconda3/envs/Germanium_Spins_FE/lib/python3.10/site-packages/ffcx/codegeneration -I/home/oliver/miniconda3/envs/Germanium_Spins_FE/include/python3.10 -c libffcx_expressions_1893f83fc25ef35f46772ddc459a44d32c1cf974.c -o ./libffcx_expressions_1893f83fc25ef35f46772ddc459a44d32c1cf974.o -O2 -g0
INFO:root:/hom

In [113]:
A = ufl.as_tensor([[1j,0],[0,1j]])

In [118]:
any(type(a)==np.complex128 for a in A.ravel())

AttributeError: 'ListTensor' object has no attribute 'ravel'

In [123]:
np.array(A)

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

AttributeError: 'ListTensor' object has no attribute 'elements'